# UN_concentration Code
#### Author: Rachel Veenstra
#### Created: 04-12-2019

In [52]:
# Importing necessary modules

import os
import math
import pandas as pd
import numpy as np
import glob
from itertools import cycle
import matplotlib.pyplot as plt
from scipy import stats
import shutil



# Identifying directory of lab sheets for quality screening

path = str(glob.os.getcwd())

user = path.split('\\')[2]

folder = "Lab_Sheets/"

datasheets = '/Users/' + user + '/Desktop/Coding/StandardLab/' + folder

glob.os.chdir(datasheets)



# Pulling data to navigate to completed folders in each topic folder

topic_folders = []

for x, folder, file in os.walk(datasheets):
    for loc in folder:
        topic_folders.append(loc)
        
topic_folders = [x for x in topic_folders if x != 'Completed']
topic_folders

for i in topic_folders:
    print(i)
    
    nit_id = []
    nit_conc = []
    
    
    columns = ['Sample_ID']
    
    nitrates = pd.DataFrame(columns = columns)
    
    location = "Student_Lists/Completed/"
    
    sheet = i + '.csv'
    
    id_file = '/Users/' + user + '/Desktop/Coding/StandardLab/' + location
    
    glob.os.chdir(id_file)
    
    id_list = pd.read_csv(sheet)

    print(id_list)
    
    # create list from id_list and append to id column in 'nitrates'
    
    screened_data = datasheets + i + '/Completed'
    
    glob.os.chdir(screened_data)

    alldata = glob.glob(screened_data + "/*.csv")
    
    for loc in alldata:
        analysis = (loc.split('_')[-7])
        flag = (loc.split('_')[-11])
        
        if flag == 'BAD':
            print("BAD")
        
        elif analysis == "NIT":
            
            print(loc)
            DF = pd.read_csv(loc)
            #print(DF)
    
            nit_df = DF[['Sample_ID', 'Absorbance']]

            nit_g = nit_df.groupby('Sample_ID')

            nit_final=nit_df.merge(nit_g.mean(),on='Sample_ID')

            nit_final.rename(columns = {'Absorbance_x':'Each', 'Absorbance_y':'Mean'}, inplace=True)
            
            nit_final['Type'] = DF['Type']
            
            nit_final['Sample_Wt(g)'] = DF['Sample_Wt(g)']
            
            nit_final['Sample_Wt(g)'].replace(np.nan, 0.3, inplace= True)
                 
            c=[]
            m=[]
            
            for i, row in nit_final.iterrows():
                
                 if row['Type'] == 'C':
            
                     c.append(float(row['Sample_ID']))
                     m.append(float(row['Mean']))
                    
                    
            slope, intercept, r_value, p_value, std_err = stats.linregress(m, c)
            
            conc = []
            
            for i, row in nit_final.iterrows():
                
                conc.append((float(row['Mean'])*slope + intercept)/(row['Sample_Wt(g)']/7.5))
            
            nit_final['Sample_Conc(micrmol/g)'] = conc
            
            for i, row in nit_final.iterrows():
                
                 if row['Type'] == 'O':
            
                     nit_id.append(row['Sample_ID'])
                     nit_conc.append(row['Sample_Conc(micrmol/g)'])
            
            columns = ['Sample_ID', 'Conc']
            
            data = pd.DataFrame(columns = columns)
            
            data['Sample_ID'] = nit_id
            data['Conc'] = nit_conc
            
            nitrates.merge(data, on='Sample_ID')
            
    print(nitrates)

LMR_IA_USB


FileNotFoundError: File b'LMR_IA_USB.csv' does not exist